In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00a 0:00:01


In [ ]:
# finetune

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP", use_auth_token=True)

/opt/conda/lib/python3.7/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/minoosh--IEMOCAP-fc9368de65fdeaef/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 10039
    })
})

In [ ]:
#from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [6]:
from datasets import load_metric

metric = load_metric("accuracy")

In [7]:
labels = dataset["train"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["9"]

'dis'

In [8]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset["train"])-1)
    example = dataset["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["emotion"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: xxx
Shape: (72159,), sampling rate: 16000



Label: sad
Shape: (36679,), sampling rate: 16000



Label: neu
Shape: (141279,), sampling rate: 16000



Label: ang
Shape: (165106,), sampling rate: 16000



Label: fru
Shape: (164484,), sampling rate: 16000


In [9]:
model_checkpoint = "facebook/wav2vec2-base"

In [10]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [11]:
max_duration = 10.0

In [12]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [13]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset

  0%|          | 0/11 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


DatasetDict({
    train: Dataset({
        features: ['emotion', 'input_values'],
        num_rows: 10039
    })
})

In [14]:
encoded_dataset = encoded_dataset.rename_column("emotion", "label")

In [15]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 10039
    })
})

In [16]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

/opt/conda/lib/python3.7/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['project_hid.weight', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.weight', 'classifier.weight', 'classifi

In [17]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 16
args = TrainingArguments(
    f"{model_name}-finetuned-ie",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [18]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [19]:
splited_encoded_dataset = encoded_dataset['train'].train_test_split(test_size=0.2)
splited_encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 8031
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 2008
    })
})

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=splited_encoded_dataset['train'],
    eval_dataset=splited_encoded_dataset['test'],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/minoosh/wav2vec2-base-finetuned-ie into local empty directory.


Download file pytorch_model.bin:   0%|          | 7.44k/361M [00:00<?, ?B/s]

Download file runs/Feb07_07-23-08_8fdbcd829a79/events.out.tfevents.1675754906.8fdbcd829a79.125.5: 100%|#######…

Download file runs/Feb07_07-23-08_8fdbcd829a79/events.out.tfevents.1675754813.8fdbcd829a79.125.2: 100%|#######…

Download file runs/Feb06_15-21-20_a29a6a7e3e1f/events.out.tfevents.1675697024.a29a6a7e3e1f.2063.0: 100%|######…

...

Clean file training_args.bin:  31%|###       | 1.00k/3.23k [00:00<?, ?B/s]

Clean file runs/Feb07_11-24-27_7d33e1eacfd4/1675774708.781824/events.out.tfevents.1675774708.7d33e1eacfd4.23.3…

Download file runs/Feb07_11-24-27_7d33e1eacfd4/1675769220.1057158/events.out.tfevents.1675769220.7d33e1eacfd4.…

Clean file runs/Feb07_11-24-27_7d33e1eacfd4/1675769220.1057158/events.out.tfevents.1675769220.7d33e1eacfd4.23.…

Clean file pytorch_model.bin:   0%|          | 1.00k/361M [00:00<?, ?B/s]

In [21]:
import torch
torch.cuda.empty_cache()

In [22]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8031
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 620
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
0,2.004400,1.936894,0.304781
1,1.802000,1.777448,0.327689
2,1.778300,1.696488,0.352092
3,1.635300,1.665670,0.348606
4,1.575900,1.569414,0.412849
5,1.485900,1.567584,0.419323
6,1.412800,1.536947,0.428785
7,1.379000,1.530918,0.424303
8,1.343400,1.535542,0.431773


***** Running Evaluation *****
  Num examples = 2008
  Batch size = 32
Saving model checkpoint to wav2vec2-base-finetuned-ie/checkpoint-62
Configuration saved in wav2vec2-base-finetuned-ie/checkpoint-62/config.json
Model weights saved in wav2vec2-base-finetuned-ie/checkpoint-62/pytorch_model.bin
Feature extractor saved in wav2vec2-base-finetuned-ie/checkpoint-62/preprocessor_config.json
Feature extractor saved in wav2vec2-base-finetuned-ie/preprocessor_config.json
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2008
  Batch size = 32
Saving model checkpoint to wav2vec2-base-finetuned-ie/checkpoint-124
Configuration saved in wav2vec2-base-finetuned-ie/checkpoint-124/config.json
Model weights saved in wav2vec2-base-fine

KeyboardInterrupt: 

In [23]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2008
  Batch size = 32


{'eval_loss': 1.5355424880981445, 'eval_accuracy': 0.43177290836653387}

In [24]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-base-finetuned-ie
Configuration saved in wav2vec2-base-finetuned-ie/config.json
Model weights saved in wav2vec2-base-finetuned-ie/pytorch_model.bin
Feature extractor saved in wav2vec2-base-finetuned-ie/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 32.0k/361M [00:00<?, ?B/s]

Upload file runs/Feb08_08-50-04_44999cd99789/events.out.tfevents.1675846349.44999cd99789.23.0: 100%|##########…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/minoosh/wav2vec2-base-finetuned-ie
   0b086a6..f67aaa1  main -> main

Dropping the following result as it does not have all the necessary fields:
{}
To https://huggingface.co/minoosh/wav2vec2-base-finetuned-ie
   f67aaa1..76c9bf2  main -> main



'https://huggingface.co/minoosh/wav2vec2-base-finetuned-ie/commit/f67aaa1c99e7fa281518bbd1f1519b88e209ab90'